<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=aad454dc859e8b742c9acc10c577bdb13a15ddf52adacf01d0ae68e583b2df01
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 10:03:57
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 35.04 K (0.25%)
Current PnL: -20.12 L (-13.35%)
CY Booked + Current PnL: -8.68 L (-5.76%)
-------------------
Total profit:  1.55 L
Total loss:  -21.66 L
-------------------
Total Booked + Current PnL: 18.30 L (14.8%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.09%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.62 L (61.94%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 8.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,64.0,H-MC,13.77,116081.0,15577.0,13535.0,0.86,15.50,11.66,28.96,79.0,1.15,0.82,54.73,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.35,83362.0,-17415.0,17506.0,0.15,-17.28,21.00,0.09,245.0,-0.99,0.59,9.52,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.63,48.0,H-MC,2.93,189290.0,24620.0,18702.0,0.01,14.95,9.88,26.31,89.0,1.32,1.34,24.13,X40N,NTT,AC
19,CIPLA,1662.74,-21.84,46.0,H-LC,3.92,208404.0,3904.0,19382.0,0.54,1.91,9.30,11.39,10.0,0.20,1.48,11.03,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,52.0,H-LC,3.34,216372.0,6006.0,22784.0,0.87,2.86,10.53,13.68,37.0,0.26,1.54,19.85,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-45.34,41.0,M-SC,1.04,92378.0,880.0,77875.0,4.09,0.96,84.30,86.07,223.0,0.01,0.66,34.15,XR,NTT,DURABLES
55,RAJOOENG,143.10,-38.56,49.0,H-SC,16.21,93000.0,-9500.0,50099.0,2.72,-9.27,53.87,39.61,114.0,-0.19,0.66,13.58,AR,ATH,MISC
44,JIOFIN,387.00,-0.66,56.0,H-LC,12.76,269825.0,2460.0,63004.0,2.45,0.92,23.35,24.48,48.0,0.04,1.92,56.40,XY24,BTT,FINANCE
33,HONAUT,58357.33,-22.75,45.0,H-SC,2.26,108810.0,-19128.0,66265.0,2.17,-14.95,60.90,36.84,143.0,-0.29,0.77,11.39,X40N,ATH,ELECTRICAL
58,REPCOHOME,880.00,-59.76,52.0,H-SC,3.51,228660.0,-57845.0,315185.0,1.98,-20.19,137.84,89.82,134.0,-0.18,1.62,20.98,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,62.28,62.0,H-SC,6.65,148318.0,-30753.0,96481.0,-1.88,-17.17,65.05,36.71,135.0,-0.32,1.05,47.16,XR,NTT,BANKS
64,SHALBY,327.00,1228.55,65.0,M-SC,20.86,180630.0,-513.0,45988.0,-1.64,-0.28,25.46,25.10,235.0,-0.01,1.28,44.76,XY24,NTT,HEALTHCARE
27,HAPPSTMNDS,1488.71,-23.48,21.0,H-SC,11.77,82680.0,-45485.0,155513.0,-1.42,-35.49,188.09,85.85,132.0,-0.29,0.59,0.00,AR,ATH,IT
78,UJJIVANSFB,60.00,106.53,63.0,H-SC,13.47,130356.0,-12123.0,31650.0,-1.25,-8.51,24.28,13.70,163.0,-0.38,0.93,53.86,OX40N,NTT,BANKS
38,INDIGOPNTS,1408.00,102.01,37.0,M-SC,2.09,132432.0,-42127.0,42166.0,-1.22,-24.13,31.84,0.02,221.0,-1.00,0.94,15.19,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1228.55,65.0,M-SC,20.86,180630.0,-513.0,45988.0,-1.64,-0.28,25.46,25.10,235.0,-0.01,1.28,44.76,XY24,NTT,HEALTHCARE
85,WHIRLPOOL,2270.0,-45.34,41.0,M-SC,1.04,92378.0,880.0,77875.0,4.09,0.96,84.30,86.07,223.0,0.01,0.66,34.15,XR,NTT,DURABLES
35,ICICIPRULI,790.0,-20.54,52.0,H-MC,2.03,136685.0,897.0,41853.0,0.72,0.66,30.62,31.48,107.0,0.02,0.97,12.86,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,47.0,M-LC,5.70,151206.0,261.0,109201.0,0.72,0.17,72.22,72.51,53.0,0.00,1.07,6.21,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,63.0,H-SC,13.47,130356.0,-12123.0,31650.0,-1.25,-8.51,24.28,13.70,163.0,-0.38,0.93,53.86,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,2.11,225162.0,-44505.0,80833.0,-0.32,-16.50,35.90,13.47,138.0,-0.55,1.60,14.49,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.35,83362.0,-17415.0,17506.0,0.15,-17.28,21.00,0.09,245.0,-0.99,0.59,9.52,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,41.0,H-SC,1.51,141542.0,-34361.0,76376.0,0.51,-19.53,53.96,23.89,149.0,-0.45,1.00,22.24,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.55,42.0,H-MC,6.23,102078.0,-29757.0,71985.0,-0.40,-22.57,70.52,32.03,98.0,-0.41,0.72,14.34,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,54.0,M-MC,12.40,229353.0,4391.0,163253.0,0.43,1.95,71.18,74.53,192.0,0.03,1.63,34.83,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,67.0,M-LC,6.23,213389.0,8313.0,104262.0,-0.19,4.05,48.86,54.89,52.0,0.08,1.51,29.13,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.63,48.0,H-MC,2.93,189290.0,24620.0,18702.0,0.01,14.95,9.88,26.31,89.0,1.32,1.34,24.13,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,49.0,H-MC,1.43,205650.0,13908.0,23855.0,0.31,7.25,11.60,19.69,99.0,0.58,1.46,14.54,XY25,NTT,AC
17,CAMS,5211.76,-4.05,53.0,H-SC,1.78,109080.0,7076.0,36847.0,1.84,6.94,33.78,43.06,122.0,0.19,0.77,25.24,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,64.0,H-MC,13.77,116081.0,15577.0,13535.0,0.86,15.50,11.66,28.96,79.0,1.15,0.82,54.73,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,70.0,H-MC,10.83,194480.0,14672.0,97746.0,0.33,8.16,50.26,62.52,88.0,0.15,1.38,45.03,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,67.0,M-LC,6.23,213389.0,8313.0,104262.0,-0.19,4.05,48.86,54.89,52.0,0.08,1.51,29.13,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.64,37.0,H-SC,9.36,126426.0,3090.0,128537.0,0.40,2.51,101.67,106.72,119.0,0.02,0.90,26.47,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-45.34,41.0,M-SC,1.04,92378.0,880.0,77875.0,4.09,0.96,84.30,86.07,223.0,0.01,0.66,34.15,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.00,32.0,H-LC,9.94,281039.0,-34803.0,147461.0,0.35,-11.02,52.47,35.67,5.0,-0.24,1.99,1.31,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.64,35.0,H-LC,5.77,207821.0,-43947.0,97156.0,0.29,-17.46,46.75,21.13,27.0,-0.45,1.47,11.79,X40,ATH,PAINTS
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,279086.0,-66870.0,135106.0,-0.65,-19.33,48.41,19.73,1.0,-0.49,1.98,2.79,X40,ATH,IT
42,ITC,452.00,-39.19,43.0,H-LC,2.13,196529.0,-3609.0,24507.0,0.29,-1.80,12.47,10.44,4.0,-0.15,1.39,3.66,X40,NTT,FMCG
19,CIPLA,1662.74,-21.84,46.0,H-LC,3.92,208404.0,3904.0,19382.0,0.54,1.91,9.30,11.39,10.0,0.20,1.48,11.03,X40N,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.60,55.0,H-LC,1.99,161810.0,-24285.0,71763.0,-0.48,-13.05,44.35,25.51,15.0,-0.34,1.15,19.24,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,43.0,H-LC,2.13,196529.0,-3609.0,24507.0,0.29,-1.80,12.47,10.44,4.0,-0.15,1.39,3.66,X40,NTT,FMCG
48,LTIM,7230.2,-3.76,55.0,H-LC,2.39,238432.0,-7719.0,86932.0,0.46,-3.14,36.46,32.18,16.0,-0.09,1.69,31.78,X200,ATH,IT
51,NESTLEIND,1377.0,-9.88,53.0,H-LC,3.07,277477.0,12051.0,44729.0,0.35,4.54,16.12,21.40,11.0,0.27,1.97,11.49,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,52.0,H-LC,3.34,216372.0,6006.0,22784.0,0.87,2.86,10.53,13.68,37.0,0.26,1.54,19.85,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.00,32.0,H-LC,9.94,281039.0,-34803.0,147461.0,0.35,-11.02,52.47,35.67,5.0,-0.24,1.99,1.31,X40N,ATH,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,279086.0,-66870.0,135106.0,-0.65,-19.33,48.41,19.73,1.0,-0.49,1.98,2.79,X40,ATH,IT
42,ITC,452.00,-39.19,43.0,H-LC,2.13,196529.0,-3609.0,24507.0,0.29,-1.80,12.47,10.44,4.0,-0.15,1.39,3.66,X40,NTT,FMCG
30,HCLTECH,1937.36,-1.94,48.0,H-LC,9.87,224384.0,-17540.0,79791.0,0.81,-7.25,35.56,25.73,8.0,-0.22,1.59,6.03,X40,ATH,IT
80,UNITDSPR,1644.00,-11.23,57.0,H-LC,6.88,232785.0,-1359.0,49979.0,-0.56,-0.58,21.47,20.77,86.0,-0.03,1.65,6.30,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.60,55.0,H-LC,1.99,161810.0,-24285.0,71763.0,-0.48,-13.05,44.35,25.51,15.0,-0.34,1.15,19.24,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,43.0,H-LC,2.13,196529.0,-3609.0,24507.0,0.29,-1.80,12.47,10.44,4.0,-0.15,1.39,3.66,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-17.64,35.0,H-LC,5.77,207821.0,-43947.0,97156.0,0.29,-17.46,46.75,21.13,27.0,-0.45,1.47,11.79,X40,ATH,PAINTS
19,CIPLA,1662.74,-21.84,46.0,H-LC,3.92,208404.0,3904.0,19382.0,0.54,1.91,9.30,11.39,10.0,0.20,1.48,11.03,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,52.0,H-LC,3.34,216372.0,6006.0,22784.0,0.87,2.86,10.53,13.68,37.0,0.26,1.54,19.85,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,279086.0,-66870.0,135106.0,-0.65,-19.33,48.41,19.73,1.0,-0.49,1.98,2.79,X40,ATH,IT
40,INFY,2275.00,-19.60,47.0,H-LC,8.08,313153.0,265.0,171420.0,-0.39,0.08,54.74,54.87,3.0,0.00,2.22,6.74,X40,BTT,IT
42,ITC,452.00,-39.19,43.0,H-LC,2.13,196529.0,-3609.0,24507.0,0.29,-1.80,12.47,10.44,4.0,-0.15,1.39,3.66,X40,NTT,FMCG
83,VBL,671.64,-22.00,32.0,H-LC,9.94,281039.0,-34803.0,147461.0,0.35,-11.02,52.47,35.67,5.0,-0.24,1.99,1.31,X40N,ATH,FMCG
1,ABB,7934.00,-38.89,58.0,H-LC,7.48,255288.0,-6331.0,125551.0,1.93,-2.42,49.18,45.57,7.0,-0.05,1.81,8.63,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,31.64,79354.0,-34195.0,74244.0,0.24,-30.11,93.56,35.27,268.0,-0.46,0.56,94.22,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,48.0,H-SC,11.50,92550.0,-5430.0,27034.0,0.11,-5.54,29.21,22.05,152.0,-0.20,0.66,34.53,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,64.0,H-MC,13.77,116081.0,15577.0,13535.0,0.86,15.50,11.66,28.96,79.0,1.15,0.82,54.73,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,63.0,H-SC,13.47,130356.0,-12123.0,31650.0,-1.25,-8.51,24.28,13.70,163.0,-0.38,0.93,53.86,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.56,146151.0,-3069.0,147247.0,1.53,-2.06,100.75,96.62,208.0,-0.02,1.04,58.17,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.56,146151.0,-3069.0,147247.0,1.53,-2.06,100.75,96.62,208.0,-0.02,1.04,58.17,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,62.0,H-SC,6.65,148318.0,-30753.0,96481.0,-1.88,-17.17,65.05,36.71,135.0,-0.32,1.05,47.16,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,67.0,M-LC,6.23,213389.0,8313.0,104262.0,-0.19,4.05,48.86,54.89,52.0,0.08,1.51,29.13,X5K,ATH,METALS
26,GREENPANEL,537.00,230.10,58.0,M-SC,3.66,147400.0,-35678.0,114662.0,-0.05,-19.49,77.79,43.14,230.0,-0.31,1.05,38.65,XY24,NTT,MISC
11,BANKINDIA,190.00,-24.22,70.0,H-MC,10.83,194480.0,14672.0,97746.0,0.33,8.16,50.26,62.52,88.0,0.15,1.38,45.03,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.09
1,25,43.28
2,50,74.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.20
MC    23.70
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.54
X40N     11.20
XR       11.00
XY25     10.43
AR        8.24
OX40N     7.70
X200      1.69
MH        1.61
X5K       1.51
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.97
H-LC    25.66
H-MC    20.46
M-SC    13.69
M-LC     5.52
M-MC     2.94
L-SC     1.41
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.78,-5.72,40.56
IT,12.61,-18.23,82.27
FINANCE,10.52,-13.92,68.93
MISC,6.95,-17.09,81.97
BANKS,6.35,-11.58,70.28
PAINTS,5.54,-18.17,35.78
ELECTRICAL,5.39,-8.31,46.31
INSURANCE,3.74,-4.75,43.90
AUTO,3.53,-15.75,71.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3221920.0,22
XR,1359278.0,14
AR,1170016.0,9
X40,788110.0,10
X40N,654628.0,9
OX40N,560900.0,10
XY25,469558.0,7
SR,260176.0,2
X5K,104262.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3708974.0,29
M-SC,1503027.0,17
H-MC,1270080.0,15
H-LC,1176367.0,15
M-LC,371957.0,4
M-MC,369682.0,2
L-SC,261247.0,3
L-MC,60108.0,1
L-LC,41020.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213133.0      6
M-SC       XY24       849395.0      7
H-SC       AR         805478.0      5
           XR         793661.0      7
H-MC       XY24       568212.0      4
H-LC       X40        507980.0      5
M-MC       XY24       369682.0      2
H-SC       X40N       304228.0      4
           SR         260176.0      2
           OX40N      259151.0      4
H-LC       X40N       216822.0      3
H-MC       X40        208048.0      4
H-LC       AR         197314.0      2
H-MC       XY25       176976.0      2
L-SC       XR         172371.0      2
M-SC       AR         167224.0      2
M-LC       XY24       158494.0      2
M-SC       XY25       147247.0      1
           OX40N      140888.0      4
H-MC       X40N       133578.0      2
M-SC       XR         126191.0      2
M-LC       XR         109201.0      1
H-LC       XY25       104315.0      3
M-LC       X5K        104262.0      1
H-MC       XR          97746.0      1
L-SC       OX40N       88876.0      1
H-LC       X200        86932.0      1
H-SC       MH          73147.0      1
M-SC       X40         72082.0      1
H-MC       OX40N       71985.0      1
H-LC       XY24        63004.0      1
L-MC       XR          60108.0      1
L-LC       XY25        41020.0      1
H-MC       MH          13535.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
